In [ ]:
%config IPCompleter.greedy=True
import os.path
import csv
import sys
csv.field_size_limit(sys.maxsize)
import petl as etl
from collections import OrderedDict
from azure.storage.blob import BlobServiceClient, ContainerClient, BlobClient, BlobBlock


In [16]:
import azure.storage.exceptions.ResourceNotFoundError

ModuleNotFoundError: No module named 'azure.storage.exceptions'

In [17]:
import azure.storage

In [19]:
dir(azure.storage.blob)

['AccessPolicy',
 'AccountSasPermissions',
 'Any',
 'AnyStr',
 'BlobAnalyticsLogging',
 'BlobBlock',
 'BlobClient',
 'BlobLeaseClient',
 'BlobProperties',
 'BlobSasPermissions',
 'BlobServiceClient',
 'BlobType',
 'BlockState',
 'ContainerClient',
 'ContainerEncryptionScope',
 'ContainerProperties',
 'ContainerSasPermissions',
 'ContentSettings',
 'CopyProperties',
 'CorsRule',
 'CustomerProvidedEncryptionKey',
 'Dict',
 'ExponentialRetry',
 'IO',
 'Iterable',
 'LeaseProperties',
 'LinearRetry',
 'LocationMode',
 'Metrics',
 'PageRange',
 'PartialBatchErrorException',
 'PremiumPageBlobTier',
 'PublicAccess',
 'RehydratePriority',
 'ResourceTypes',
 'RetentionPolicy',
 'SequenceNumberAction',
 'StandardBlobTier',
 'StaticWebsite',
 'StorageErrorCode',
 'StorageStreamDownloader',
 'Union',
 'UserDelegationKey',
 'VERSION',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_blob_clie

In [3]:
account_name='datalake2dictas'
account_key='wlxJHcWvtVhPpL/zs6l+F1bJGKZnJ4HppZcVyh+ns32oH46E3dY/HBLau3V6um9hv+KZf/3mXEAL5nHD41X3jg=='
container_from = 'transient'
container_to = 'raw'
upload_file = '/opt/jupyter_etl_dictas/datalake/transient/beneficiario/beneficiarios_atualizado.csv'
_LOCALDIR = '/opt/jupyter_etl_dictas/datalake/transient'

In [4]:
cc_transient = ContainerClient(account_url=f"https://{account_name}.blob.core.windows.net/",container_name=container_from, credential=account_key)
cc_raw = ContainerClient(account_url=f"https://{account_name}.blob.core.windows.net/",container_name=container_to, credential=account_key)


In [5]:
_TEMP_FILE='datalake/temp'
_METADADOS='metadado/list.csv'
_SOURCE='17'
_PROC_FILES='datalake/proc'
_NM_ARQ = 'CUSTOS.csv'

### Carrega blob Custos

#### Verifica Arquivo no Container

In [6]:
blob_list = cc_transient.list_blobs()
for blob in blob_list:
    print(blob.name + '\n')
    

BENEFICIARIO.csv

CONSULTAS.csv

EMPRESA.csv

PRESTADORES.csv

RECEITAS.csv

SERVICOS.csv

VIDAS.csv



#### Faz Download para diretório local

In [10]:
try:
    with open (f'{_PROC_FILES}/{_SOURCE}/{_NM_ARQ}', 'wb') as data_from:
        data_from.write(cc_transient.get_blob_client(_NM_ARQ).download_blob().readall())
except StorageErrorException:
    print('ERRO')


NameError: name 'StorageErrorException' is not defined

#### Carrega dados para transformação

In [29]:
table = etl.fromcsv(f'{_PROC_FILES}/{_SOURCE}/{_NM_ARQ}', delimiter='|')
table

cod_custo,fk_beneficiario,mes_competencia,nr_guia,nr_guia_principal,dt_realizacao,fk_cid,dt_ini_internacao,dt_fim_internacao,fk_medicosolicitante,cod_executante,fk_local_atend,fk_capa_unico,fk_destino_pgto,fk_servico,cod_servico_tipo,qtd,qtd_unica,vl_unitario,vl_total,vl_coparticipacao,ind_internacao,tipo_registro,tipo_atendimento,tipo_guia,tipo_internacao,tipo_acomodacao,ds_indicacao_clinica,vl_hm,vl_co,vl_filme,partic_medico
22416924,00060378017962,082019,00000001,NULL,10/07/2019,NULL,NULL,NULL,23113,64008,001,001,114418,41301099,Procedimentos e eventos em saude,1.00,NULL,0.00,0.00,3.31,NÃO,C,EXAME,GUIA DE SP_SADT/ODONTOLOGIA,NULL,NULL,Rorina,11.03,0,0,NULL
22551681,00023530008003,082019,00000004,NULL,18/07/2019,NULL,NULL,NULL,23113,64008,001,001,114418,40901300,Procedimentos e eventos em saude,1.00,NULL,0.00,0.00,30.58,NÃO,C,EXAME,GUIA DE SP_SADT/ODONTOLOGIA,NULL,NULL,rotina,58.14,39.6,4.2,NULL
22575840,00029784009793,082019,00000005,NULL,23/07/2019,NULL,NULL,NULL,24326,64008,001,001,114418,41301099,Procedimentos e eventos em saude,1.00,NULL,0.00,0.00,3.31,NÃO,C,EXAME,GUIA DE SP_SADT/ODONTOLOGIA,NULL,NULL,Rotina,11.03,0,0,NULL
22672218,00051380015742,082019,00000001,NULL,10/07/2019,NULL,NULL,NULL,64208,NULL,001,001,013878,90173538,Medicamentos,1.00,NULL,152.00,152.00,45.60,NÃO,C,EXAME,GUIA DE SP_SADT/ODONTOLOGIA,NULL,NULL,NULL,0,152,0,Nao se Aplica
22672241,00051380015742,082019,00000001,NULL,10/07/2019,NULL,NULL,NULL,64208,NULL,001,001,013878,70762384,"Materiais e Orteses, Proteses e Materiai",1.00,NULL,3.76,3.76,1.13,NÃO,C,EXAME,GUIA DE SP_SADT/ODONTOLOGIA,NULL,NULL,NULL,0,3.76,0,Nao se Aplica


In [30]:
#Transforma sexo e tira os espaços vazios e substitui string 'NULL' por ''
def rowmapper(row):
    strnull = {'NULL':''}
    return [row[0].strip(),
            row[1].strip(),
            row[2].strip(),
            row[3].strip(),
            strnull[row['nr_guia_principal'].strip()] if row['nr_guia_principal'].strip() in strnull else row['nr_guia_principal'].strip(),
            row[5].strip(),
            strnull[row['fk_cid'].strip()] if row['fk_cid'].strip() in strnull else row['fk_cid'].strip(),
            strnull[row['dt_ini_internacao'].strip()] if row['dt_ini_internacao'].strip() in strnull else row['dt_ini_internacao'].strip(),
            strnull[row['dt_fim_internacao'].strip()] if row['dt_fim_internacao'].strip() in strnull else row['dt_fim_internacao'].strip(),
            strnull[row['fk_medicosolicitante'].strip()] if row['fk_medicosolicitante'].strip() in strnull else row['fk_medicosolicitante'].strip(),
            strnull[row['cod_executante'].strip()] if row['cod_executante'].strip() in strnull else row['cod_executante'].strip(),
            row[11].strip(),
            row[12].strip(),
            row[13].strip(),
            row[14].strip(),
            row[15].strip(),
            row[16].strip(),
            strnull[row['qtd_unica'].strip()] if row['qtd_unica'].strip() in strnull else row['qtd_unica'].strip(),
            row[18].strip(),
            row[19].strip(),
            row[20].strip(),
            row[21].strip(),
            row[22].strip(),
            row[23].strip(),
            row[24].strip(),
            strnull[row['tipo_internacao'].strip()] if row['tipo_internacao'].strip() in strnull else row['tipo_internacao'].strip(),
            strnull[row['tipo_acomodacao'].strip()] if row['tipo_acomodacao'].strip() in strnull else row['tipo_acomodacao'].strip(),
            strnull[row['ds_indicacao_clinica'].strip()] if row['ds_indicacao_clinica'].strip() in strnull else row['ds_indicacao_clinica'].strip(),
            row[28].strip(),
            row[29].strip(),
            row[30].strip(),
            strnull[row['partic_medico'].strip()] if row['partic_medico'].strip() in strnull else row['partic_medico'].strip()
            ]

table1 = etl.rowmap(table, rowmapper,
                    header=['cod_custo','fk_beneficiario','mes_competencia','nr_guia','nr_guia_principal',
                            'dt_realizacao','fk_cid','dt_ini_internacao','dt_fim_internacao','fk_medicosolicitante',
                            'cod_executante','fk_local_atend','fk_capa_unico','fk_destino_pgto','fk_servico',
                            'cod_servico_tipo','qtd','qtd_unica','vl_unitario','vl_total','vl_coparticipacao',
                            'ind_internacao','tipo_registro','tipo_atendimento','tipo_guia','tipo_internacao',
                            'tipo_acomodacao','ds_indicacao_clinica','vl_hm','vl_co','vl_filme','partic_medico'
                           ])
table1

cod_custo,fk_beneficiario,mes_competencia,nr_guia,nr_guia_principal,dt_realizacao,fk_cid,dt_ini_internacao,dt_fim_internacao,fk_medicosolicitante,cod_executante,fk_local_atend,fk_capa_unico,fk_destino_pgto,fk_servico,cod_servico_tipo,qtd,qtd_unica,vl_unitario,vl_total,vl_coparticipacao,ind_internacao,tipo_registro,tipo_atendimento,tipo_guia,tipo_internacao,tipo_acomodacao,ds_indicacao_clinica,vl_hm,vl_co,vl_filme,partic_medico
22416924,00060378017962,082019,00000001,,10/07/2019,,,,23113,64008,001,001,114418,41301099,Procedimentos e eventos em saude,1.00,,0.00,0.00,3.31,NÃO,C,EXAME,GUIA DE SP_SADT/ODONTOLOGIA,,,Rorina,11.03,0,0,
22551681,00023530008003,082019,00000004,,18/07/2019,,,,23113,64008,001,001,114418,40901300,Procedimentos e eventos em saude,1.00,,0.00,0.00,30.58,NÃO,C,EXAME,GUIA DE SP_SADT/ODONTOLOGIA,,,rotina,58.14,39.6,4.2,
22575840,00029784009793,082019,00000005,,23/07/2019,,,,24326,64008,001,001,114418,41301099,Procedimentos e eventos em saude,1.00,,0.00,0.00,3.31,NÃO,C,EXAME,GUIA DE SP_SADT/ODONTOLOGIA,,,Rotina,11.03,0,0,
22672218,00051380015742,082019,00000001,,10/07/2019,,,,64208,,001,001,013878,90173538,Medicamentos,1.00,,152.00,152.00,45.60,NÃO,C,EXAME,GUIA DE SP_SADT/ODONTOLOGIA,,,,0,152,0,Nao se Aplica
22672241,00051380015742,082019,00000001,,10/07/2019,,,,64208,,001,001,013878,70762384,"Materiais e Orteses, Proteses e Materiai",1.00,,3.76,3.76,1.13,NÃO,C,EXAME,GUIA DE SP_SADT/ODONTOLOGIA,,,,0,3.76,0,Nao se Aplica


#### Add fk_empresa

In [31]:
table2 = etl.addfields(table1,[('cd_empresa',_SOURCE)])
#table2
etl.tocsv(table2, f'{_PROC_FILES}/{_SOURCE}/t{_NM_ARQ}', delimiter='|')

In [32]:
upload_file=f'{_PROC_FILES}/{_SOURCE}/t{_NM_ARQ}'
#print(upload_file)
try:
    with open (upload_file, "rb") as data :
        #print(data)
        cc_raw.upload_blob(_NM_ARQ, data, overwrite = True)
finally:
    print("Tudo carregado")

Tudo carregado


#### Verifica se arquivo está no Blob

In [33]:
blob_list = cc_raw.list_blobs()
for blob in blob_list:
    print(blob.name + '\n')

BENEFICIARIO.csv

CONSULTAS.csv

CUSTOS.csv

DIARIAS.csv

EMPRESA.csv

PRESTADORES.csv

SERVICOS.csv

